<a href="https://colab.research.google.com/github/Shreyes45/BIS-lab/blob/main/CIE-lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import math

def levy_flight(beta):
    sigma = (math.gamma(1+beta) * math.sin(math.pi * beta / 2) /
             (math.gamma((1+beta)/2) * beta * 2**((beta-1)/2))) ** (1/beta)
    u = np.random.normal(0, sigma, size=2)
    v = np.random.normal(0, 1, size=2)
    return u / (np.abs(v) ** (1/beta))

def material_weight_strength(vars):
    t, rho = vars
    if t <= 0 or rho <= 0:
        return 1e9
    strength = 500/t
    weight = rho * t
    penalty = 0
    if strength < 50:
        penalty += (50 - strength)**2
    return weight + penalty

def cuckoo_search(n=20, pa=0.25, iterations=200):
    lb = np.array([0.5, 1000])
    ub = np.array([5.0, 5000])
    nests = lb + (ub - lb) * np.random.rand(n, 2)
    fitness = np.array([material_weight_strength(x) for x in nests])
    best = nests[np.argmin(fitness)]

    for _ in range(iterations):
        new_nests = np.copy(nests)
        for i in range(n):
            step = levy_flight(1.5)
            new_nests[i] = nests[i] + step * (nests[i] - best)
            new_nests[i] = np.clip(new_nests[i], lb, ub)
        new_fitness = np.array([material_weight_strength(x) for x in new_nests])
        improved = new_fitness < fitness
        nests[improved] = new_nests[improved]
        fitness[improved] = new_fitness[improved]
        K = np.random.rand(n, 2) > pa
        random_nests = lb + (ub - lb) * np.random.rand(n, 2)
        nests = nests * K + random_nests * (~K)
        fitness = np.array([material_weight_strength(x) for x in nests])
        best = nests[np.argmin(fitness)]

    return best, material_weight_strength(best)

np.set_printoptions(suppress=True, floatmode='fixed', precision=6)

best_solution, best_value = cuckoo_search()

print("Optimal Solution Found:")
print(f"Thickness (t): {best_solution[0]}")
print(f"Density (rho): {best_solution[1]}")
print(f"Minimum Fitness Value: {best_value}")


Optimal Solution Found:
Thickness (t): 0.5
Density (rho): 1000.0
Minimum Fitness Value: 500.0
